In [25]:
# range of dist

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

In [2]:
fpath = 'data/prep/'
dfile = 'form_data_v2.2.csv'

In [3]:
df = pd.read_csv( fpath + dfile, header=0, parse_dates=True )

In [4]:
# flat races
df = df[df.Type.isna()]

In [5]:
# remove entries with missing Draw - some of these could be imputed or manually entered
# print(df.RaceId[df.Draw.isna()].unique())
df = df[~df.Draw.isna()]

In [6]:
# no. declared from max draw by race, ie FieldSize + RR + withdrawn
DeclFieldSize = df[['RaceId', 'Draw']].groupby('RaceId').max().squeeze().rename('DeclFieldSize').astype(int)
df = df.join(DeclFieldSize, on='RaceId')

In [7]:
# rank of draw by race
df['DrawRank'] = df[['RaceId', 'Draw']].groupby('RaceId').rank('first').astype(int)

In [8]:
# no. stalls from max draw rank by race ( ==FieldSize ?? == grouby count ?? )
N_Stalls = df[['RaceId', 'DrawRank']].groupby('RaceId').max().squeeze().rename('N_Stalls').astype(int)
df = df.join(N_Stalls, on='RaceId')

In [9]:
# df[df.FieldSize != df.N_Stalls][['RaceId','FieldSize','DeclFieldSize','N_Stalls']]

In [10]:
# replace non-numeric Posn with last place and cast to int
df['Posn'] = df.apply(lambda r: int(r.Posn) if r.Posn.isnumeric() else r.DeclFieldSize, axis=1)

In [11]:
g = df[df.Posn<=3].groupby('RaceId')['DrawRank']
a1 = g.max() - g.min() < 3

In [12]:
# create sim DF with any EW value boolean
df['EWRet'] = df.ReturnPlace>=3.0
a2 = df[df.Posn<=3].groupby('RaceId')['EWRet'].any()

In [13]:
# create DF with RaceId, CourseName, Distance to nearest 110 yds, Going (≤Gd,>Gd)
b = df[['RaceId','CourseName','Distance','Going1N']].drop_duplicates()
b['Distance_f'] = np.round(b['Distance'] / 110) / 2.0
b['Going'] = df.Going1N <= 3.0
b.drop(['Distance','Going1N'], axis=1, inplace=True)

In [14]:
# merge with a1
c = a1.reset_index().merge(b, on='RaceId').rename({'DrawRank':'DrawPaceBias'},axis=1)

In [15]:
# merge with a2
c = a2.reset_index().merge(c, on='RaceId')

In [16]:
# groupby Course, Distance, Going ; mean(a) ie propn

d = c.drop('RaceId', axis=1).groupby(['CourseName','Distance_f','Going'])

In [17]:
e = d.mean().reset_index().merge(d.count()['DrawPaceBias'].reset_index().\
                            rename({'DrawPaceBias':'Count'},axis=1),on=['CourseName','Distance_f','Going']).\
                            sort_values('DrawPaceBias',ascending=False).\
                            reset_index(drop=True).reset_index(names='Rank')

In [18]:
f = e[e.Count>=5]

In [19]:
f.groupby('CourseName')[['DrawPaceBias','EWRet']].mean().sort_values('DrawPaceBias',ascending=False)

,DrawPaceBias,EWRet
CourseName,,
Tramore,0.579641,0.443086
Ffos Las,0.203113,0.486874
Salisbury,0.201894,0.576815
Newmarket,0.197294,0.556220
Thurles,0.195139,0.714583
Killarney,0.191759,0.649960
Clonmel,0.189583,0.720833
Yarmouth,0.181801,0.589646
Ripon,0.179127,0.566410


In [97]:
f[(f.CourseName=='Brighton')&f.Going].sort_values('DrawPaceBias',ascending=False)

,Rank,CourseName,Distance_f,Going,EWRet,DrawPaceBias,Count
86,86,Brighton,12.0,True,0.480000,0.250000,100
120,120,Brighton,8.0,True,0.523810,0.222222,189
193,193,Brighton,10.0,True,0.447368,0.197368,152
215,215,Brighton,5.5,True,0.394737,0.184211,152
286,286,Brighton,7.0,True,0.584416,0.160173,231
467,467,Brighton,6.0,True,0.502857,0.120000,175


In [99]:
f[(f.CourseName=='Chepstow')&(f.Going)].sort_values('DrawPaceBias',ascending=False)

,Rank,CourseName,Distance_f,Going,EWRet,DrawPaceBias,Count
131,131,Chepstow,10.0,True,0.390625,0.218750,64
233,233,Chepstow,12.0,True,0.467742,0.177419,62
246,246,Chepstow,8.0,True,0.505376,0.172043,93
307,307,Chepstow,7.0,True,0.604839,0.153226,124
319,319,Chepstow,5.0,True,0.526882,0.150538,93
414,414,Chepstow,6.0,True,0.669725,0.128440,109
567,567,Chepstow,16.0,True,0.482759,0.103448,29


In [98]:
f[(f.CourseName=='Wolverhampton')&(f.Going)].sort_values('DrawPaceBias',ascending=False)

,Rank,CourseName,Distance_f,Going,EWRet,DrawPaceBias,Count
269,269,Wolverhampton,16.5,True,0.640288,0.165468,139
365,365,Wolverhampton,5.0,True,0.601439,0.138129,695
389,389,Wolverhampton,12.0,True,0.591736,0.133884,605
482,482,Wolverhampton,7.0,True,0.639315,0.117223,1109
484,484,Wolverhampton,14.0,True,0.623256,0.116279,215
562,562,Wolverhampton,6.0,True,0.656620,0.104413,929
566,566,Wolverhampton,8.5,True,0.638229,0.103672,926
576,576,Wolverhampton,9.5,True,0.638060,0.100746,804


In [100]:
f[(f.CourseName=='Windsor')&(f.Going)].sort_values('DrawPaceBias',ascending=False)

,Rank,CourseName,Distance_f,Going,EWRet,DrawPaceBias,Count
143,143,Windsor,8.5,True,0.607143,0.214286,28
243,243,Windsor,11.5,True,0.519553,0.173184,179
336,336,Windsor,5.0,True,0.570000,0.145000,200
402,402,Windsor,8.0,True,0.552764,0.130653,199
428,428,Windsor,10.0,True,0.596413,0.125561,223
487,487,Windsor,6.0,True,0.632948,0.115607,346


In [101]:
f[(f.CourseName=='Chepstow')&(f.Going)].sort_values('DrawPaceBias',ascending=False)

,Rank,CourseName,Distance_f,Going,EWRet,DrawPaceBias,Count
131,131,Chepstow,10.0,True,0.390625,0.218750,64
233,233,Chepstow,12.0,True,0.467742,0.177419,62
246,246,Chepstow,8.0,True,0.505376,0.172043,93
307,307,Chepstow,7.0,True,0.604839,0.153226,124
319,319,Chepstow,5.0,True,0.526882,0.150538,93
414,414,Chepstow,6.0,True,0.669725,0.128440,109
567,567,Chepstow,16.0,True,0.482759,0.103448,29


In [120]:
f[(f.CourseName=='Carlisle')&(f.Going)].sort_values('DrawPaceBias',ascending=False)

,Rank,CourseName,Distance_f,Going,EWRet,DrawPaceBias,Count
119,119,Carlisle,17.0,True,0.666667,0.222222,9
140,140,Carlisle,5.0,True,0.660714,0.214286,56
251,251,Carlisle,9.0,True,0.658537,0.170732,41
370,370,Carlisle,8.0,True,0.575000,0.137500,80
395,395,Carlisle,6.0,True,0.676190,0.133333,105
465,465,Carlisle,7.0,True,0.600000,0.120000,75
497,497,Carlisle,11.0,True,0.613636,0.113636,44
855,855,Carlisle,14.0,True,0.400000,0.000000,10


In [119]:
f[(f.CourseName=='Beverley')&(f.Going)].sort_values('DrawPaceBias',ascending=False)

,Rank,CourseName,Distance_f,Going,EWRet,DrawPaceBias,Count
74,74,Beverley,12.0,True,0.507692,0.261538,65
218,218,Beverley,7.5,True,0.578261,0.182609,230
377,377,Beverley,16.0,True,0.409091,0.136364,22
419,419,Beverley,5.0,True,0.644828,0.127586,290
477,477,Beverley,8.5,True,0.627273,0.118182,110
542,542,Beverley,10.0,True,0.572327,0.106918,159


In [25]:
f[(f.CourseName=='Pontefract')&(f.Going)].sort_values('DrawPaceBias',ascending=False)

,Rank,CourseName,Distance_f,Going,EWRet,DrawPaceBias,Count
115,115,Pontefract,12.0,True,0.520000,0.226667,75
222,222,Pontefract,18.0,True,0.818182,0.181818,11
238,238,Pontefract,5.0,True,0.536082,0.175258,97
299,299,Pontefract,10.0,True,0.614458,0.156627,83
371,371,Pontefract,8.0,True,0.572519,0.137405,131
383,383,Pontefract,6.0,True,0.598958,0.135417,192
538,538,Pontefract,17.0,True,0.357143,0.107143,28


In [22]:
f.columns

Index(['Rank', 'CourseName', 'Distance_f', 'Going', 'EWRet', 'DrawPaceBias',
       'Count'],
      dtype='object')

In [109]:
df[(df.Jockey=='William Cox')&(df.CourseName=='Chepstow')&(df.ReturnPlace>0)&(df.AgeMax==2.0)]

,RaceId,CourseName,RaceDate,RaceTime,Type,Class,Prize,FieldSize,Distance,RaceWinTime,...,Going1,Going1N,Won,Placed,ReturnWin,ReturnPlace,DeclFieldSize,DrawRank,N_Stalls,EWRet
448781,43720111,Chepstow,2019-09-12,15:35:00,NaN,5.0,3429.0,11,1556,86.75,...,goodtosoft,4.0,False,0.20,0.0,5.000,11,3,11,True
515048,44012105,Chepstow,2020-06-30,19:00:00,NaN,5.0,4140.0,9,1556,85.10,...,good,3.0,True,0.20,5.0,1.800,10,2,9,False
533416,44057101,Chepstow,2020-08-14,16:35:00,NaN,5.0,4140.0,5,1556,83.71,...,good,3.0,False,0.25,0.0,2.875,6,5,5,False
800759,44803108,Chepstow,2022-08-30,13:20:00,NaN,5.0,3672.0,10,1336,69.98,...,good,3.0,False,0.20,0.0,4.600,12,1,10,True


In [105]:
df.columns

Index(['RaceId', 'CourseName', 'RaceDate', 'RaceTime', 'Type', 'Class',
       'Prize', 'FieldSize', 'Distance', 'RaceWinTime', 'Posn', 'TotalBtn',
       'HorseName', 'Draw', 'Sp', 'WeightLBS', 'Aid', 'Trainer', 'Jockey',
       'Allow', 'OR', 'Comments', 'Territory', 'RT', 'AgeMin', 'AgeMax', 'AW',
       'Going1', 'Going1N', 'Won', 'Placed', 'ReturnWin', 'ReturnPlace',
       'DeclFieldSize', 'DrawRank', 'N_Stalls', 'EWRet'],
      dtype='object')

In [118]:
pd.options.display.max_rows = 1000
df1 = df[(df.AgeMin==3.0)&(df.CourseName=='Beverley')&(df.RT=='Hc')&(df.Distance>7*220)&(df.Posn<3)]
df1[['RaceDate','Class','N_Stalls','Going1','Posn','Draw','Sp','WeightLBS','Allow','OR','Comments']]

,RaceDate,Class,N_Stalls,Going1,Posn,Draw,Sp,WeightLBS,Allow,OR,Comments
23860,2016-04-13,4.0,7,soft,1,6.0,5.000000,130,0,77.0,always prominent ridden to lead over 1f out ra...
23861,2016-04-13,4.0,7,soft,2,2.0,12.000000,121,0,68.0,tracked leaders ridden to challenge over 2f ou...
23867,2016-04-13,5.0,4,soft,1,3.0,1.870000,130,0,72.0,tracked leader every chance from over 2f out r...
23868,2016-04-13,5.0,4,soft,2,5.0,4.000000,121,0,63.0,led strongly challenged from over 2f out heade...
23871,2016-04-13,4.0,6,soft,1,2.0,3.000000,132,0,79.0,chased leaders headway over 2f out hung right ...
23872,2016-04-13,4.0,6,soft,2,1.0,3.000000,130,0,77.0,led keen hung left throughout driven over 2f o...
26472,2016-04-21,5.0,9,goodtosoft,1,1.0,6.500000,130,0,67.0,raced keenly made all ridden over 1f out soon ...
26473,2016-04-21,5.0,9,goodtosoft,2,3.0,4.500000,126,0,63.0,raced keenly tracking leading pair went 2nd en...
26481,2016-04-21,3.0,5,goodtosoft,1,3.0,2.250000,127,0,83.0,slowly into stride soon recovered tracking lea...
26482,2016-04-21,3.0,5,goodtosoft,2,5.0,2.250000,127,0,83.0,held up off the pace in last headway on rail w...


In [138]:
df.RT.str.find('Hc')>-1

365        True
366        True
367        True
368        True
369        True
           ... 
1084322    True
1084323    True
1084324    True
1084325    True
1084326    True
Name: RT, Length: 654503, dtype: bool

In [133]:
df.columns

Index(['RaceId', 'CourseName', 'RaceDate', 'RaceTime', 'Race', 'Type', 'Class',
       'Prize', 'FieldSize', 'Distance', 'RaceWinTime', 'Posn', 'TotalBtn',
       'HorseName', 'Draw', 'Sp', 'Age', 'WeightLBS', 'Aid', 'Trainer',
       'Jockey', 'Allow', 'OR', 'Comments', 'Territory', 'RT', 'AgeMin',
       'AgeMax', 'AW', 'Going1', 'Going1N', 'Won', 'Placed', 'ReturnWin',
       'ReturnPlace', 'DeclFieldSize', 'DrawRank', 'N_Stalls', 'EWRet'],
      dtype='object')

In [141]:
df[(df.CourseName=='Carlisle')&(df.RT.str.find('Hc')>-1)&(df.Distance>=8*220-55)&(df.Distance<8*220+55)&
    (df.Race.str.find('Bell')>-1)][['RaceDate','Race','Class','FieldSize',
                                                    'Draw','Posn','Sp','Comments']]

,RaceDate,Race,Class,FieldSize,Draw,Posn,Sp,Comments
51965,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,15.0,1,11.000000,made all ran on well
51966,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,13.0,2,8.000000,hedl up towards rear of middivsiopn headway 2f...
51967,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,14.0,3,10.000000,middivision headway on outside 2f out went 2nd...
51968,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,16.0,4,40.000000,started very slowly behind headway over 1f opu...
51969,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,7.0,5,14.000000,middivsion headway on outside over 1f out kept...
51970,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,6.0,6,14.000000,chased leaders ridden over 2f out no extra tow...
51971,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,3.0,7,16.000000,chased winner 2f chased leaders went 2nd again...
51972,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,9.0,8,25.000000,held up towards rear headway under pressure ov...
51973,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,17.0,9,66.000000,towards rear some late headway never threatened
51974,2016-06-22,bookies.com Carlisle Bell Consolation Race (Ha...,4.0,16,1.0,10,7.500000,chased leaders pushed along over 2f out ridden...
